In [1]:
import pandas as pd
import numpy as np

import neologdn
import MeCab

import re

from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

# bag of wordsを作成するためのライブラリ
from gensim import corpora, matutils
# TF-IDFを作成するためのライブラリ
from gensim import models

C:\Users\Tomori kengo\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


### サンプルデータの読み込み

In [2]:
#encoding='utf-8'で上手くいった
diary_df = pd.read_csv('SBI_Financial statement_201903.csv', encoding='utf-8')

In [3]:
diary_df

,text
0,国内経済が緩やかに回復するなか、マーケット環境は、米国と中国の貿易摩擦問題に対する警戒感等か...
1,このような環境の中、当社業績においては、ホールセールビジネスの拡大、トレーディング収益や金融...
2,当社は引続き他社を大きく上回る高いシェアを維持し、35.9％のシェアを獲得。
3,先物・オプションの委託個人売買代金シェアは、引き続き高水準を維持。
4,投資信託残高の四半期末残高は過去最高を更新し、信託報酬は高水準を維持。
5,2018年4月から2018年6月までの上場会社数は20社。
6,同期間のSBI証券引受関与率は100％と 引き続き業界トップ。


### MeCabにかける前準備としてneologdn.normalize()を使用して文章全体を正規化

In [4]:
def get_diary_normalization(text):
    diary_normalization = neologdn.normalize(text)
    return diary_normalization

### MeCab + neologdで形態素解析し、名詞、形容詞原形と動詞原形を抽出しリストに格納
##### 参考：https://github.com/kujirahand/book-mlearn-gyomu/blob/master/src/ch4/Doc2Vec/create_model.py

In [5]:
neologd_tagger = MeCab.Tagger('-Ochasen -d C:\mecab-ipadic-neologd')

# 引数のテキストを分かち書きして配列にする
# node.surface: 文字のみ取得できる 出力例：同  期間  の  SBI  証券
# node.feature: 品詞、原形などの詳細を取得できる:「品詞,品詞細分類1,品詞細分類2,品詞細分類3,活用形,活用型,原形,読み,発音」の順
# よって、「品詞」を取得したい場合はnode.featureをsplit()でリスト型にした後に抽出したい詳細のインデックス番号[0]を指定すればよい
# また、動詞や形容詞の「原形（の単語）」を取得したい場合はインデックス番号[6]を指定すればよい

def split_words(diary_normalization):
    node = neologd_tagger.parseToNode(diary_normalization) #parseだとエラー「 'str' object has no attribute 'feature'」
    wakati_words = []
    while node is not None:
        hinshi = node.feature.split(",")[0]
        if  hinshi in ["名詞"]:
            wakati_words.append(node.surface)
        elif hinshi in ["動詞", "形容詞"]:
            wakati_words.append(node.feature.split(",")[6])
        node = node.next
    return wakati_words

In [6]:
wakati_list = []

# 文章の上から順に作成した関数を実行
for diary in diary_df['text']:
    diary_normalization = get_diary_normalization(diary)
    wakati_words = split_words(diary_normalization)
    wakati_list.append(wakati_words)
    
wakati_list

[['国内',
  '経済',
  '緩やか',
  '回復',
  'する',
  'なか',
  'マーケット',
  '環境',
  '米国',
  '中国',
  '貿易摩擦',
  '問題',
  '警戒感',
  '等',
  '一進一退',
  '展開'],
 ['よう',
  '環境',
  '中',
  '当社',
  '業績',
  'ホールセール',
  'ビジネス',
  '拡大',
  'トレーディング',
  '収益',
  '金融',
  '収益',
  '増加',
  '前年同期',
  '比',
  '増収増益',
  '達成'],
 ['当社', '他社', '大きい', '上回る', '高い', 'シェア', '維持', 'する', '35.9%', 'シェア', '獲得'],
 ['先物', 'オプション', '委託', '個人', '売買代金', 'シェア', '高水準', '維持'],
 ['投資信託', '残高', '四半期', '末', '残高', '過去最高', '更新', 'する', '信託報酬', '高水準', '維持'],
 ['2018年', '4月', '2018年', '6月', '上場会社', '数', '20', '社'],
 ['期間', 'SBI証券', '引受', '関与', '率', '100%', '業界', 'トップ']]

### bag of Wordsの実装
##### 参考書籍：前処理大全[データ分析のためのSQL/R/Python実践テクニック] 

In [7]:
# bag of words作成準備
# 全種類の単語のIDを付与した辞書の作成
corpus_dic = corpora.Dictionary(wakati_list)

In [8]:
# 各文章の単語リストをコーパス（辞書の単語IDと単語の出現回数）リストに変換
# 辞書オブジェクトからdoc2bow関数を呼び出し、引数に単語リストを指定することで辞書に基づいたコーパスを取得できる
# コーパスとは、文章に含まれる単語を、辞書の単語IDと単語の出現回数の組み合わせに変換したもの
# bag of wordsに変換するときは変換元のデータ番号が行列の行番号に、単語IDの番号が行列の列番号に、出現回数が行列の値に該当
corpus_list = [corpus_dic.doc2bow(wakati) for wakati in wakati_list]

In [9]:
# matutilsオブジェクトのcorpus2csc関数：引数に指定したコーパスリストをスパースマトリックス(CSC型)のbag of wordsに変換する
wakati_matrix = matutils.corpus2csc(corpus_list)

In [10]:
print(wakati_matrix)# printなしの「wakati_matrix」だけだと<67x7 sparse matrix of type '<class 'numpy.float64'>'と出力

  (0, 0)	1.0
  (1, 0)	1.0
  (2, 0)	1.0
  (3, 0)	1.0
  (4, 0)	1.0
  (5, 0)	1.0
  (6, 0)	1.0
  (7, 0)	1.0
  (8, 0)	1.0
  (9, 0)	1.0
  (10, 0)	1.0
  (11, 0)	1.0
  (12, 0)	1.0
  (13, 0)	1.0
  (14, 0)	1.0
  (15, 0)	1.0
  (9, 1)	1.0
  (16, 1)	1.0
  (17, 1)	1.0
  (18, 1)	1.0
  (19, 1)	1.0
  (20, 1)	1.0
  (21, 1)	1.0
  (22, 1)	2.0
  (23, 1)	1.0
  :	:
  (0, 4)	1.0
  (37, 4)	1.0
  (44, 4)	1.0
  (45, 4)	1.0
  (46, 4)	1.0
  (47, 4)	1.0
  (48, 4)	1.0
  (49, 4)	1.0
  (50, 4)	2.0
  (51, 4)	1.0
  (52, 5)	1.0
  (53, 5)	2.0
  (54, 5)	1.0
  (55, 5)	1.0
  (56, 5)	1.0
  (57, 5)	1.0
  (58, 5)	1.0
  (59, 6)	1.0
  (60, 6)	1.0
  (61, 6)	1.0
  (62, 6)	1.0
  (63, 6)	1.0
  (64, 6)	1.0
  (65, 6)	1.0
  (66, 6)	1.0


In [11]:
print(wakati_matrix.toarray())

[[1. 0. 1. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 2. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 2. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 1. 1. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0.

### TF-IDFを利用したbag of wordsの作成
##### 参考書籍：前処理大全[データ分析のためのSQL/R/Python実践テクニック]

In [12]:
# TF-IDFモデルの作成
# models.TfidfModel関数はコーパスリストをTF-IDF値に変換するオブジェクトを生成する関数
# 引数にコーパスリストを指定し、normalize引数をTrueにするとTF-IDF値をL2ノルムによって正規化した値に変換するオブジェクトになる
tfidf_model = models.TfidfModel(corpus_list, normalize=True)

In [13]:
# corpusにTF-IDFを適用
corpus_list_tfidf = tfidf_model[corpus_list]
wakati_tfidf_matrix = matutils.corpus2csc(corpus_list_tfidf)

In [14]:
print(wakati_tfidf_matrix.toarray())

[[0.11394006 0.         0.14977713 0.         0.13253451 0.
  0.        ]
 [0.26167554 0.         0.         0.         0.         0.
  0.        ]
 [0.26167554 0.         0.         0.         0.         0.
  0.        ]
 [0.26167554 0.         0.         0.         0.         0.
  0.        ]
 [0.26167554 0.         0.         0.         0.         0.
  0.        ]
 [0.26167554 0.         0.         0.         0.         0.
  0.        ]
 [0.26167554 0.         0.         0.         0.         0.
  0.        ]
 [0.26167554 0.         0.         0.         0.         0.
  0.        ]
 [0.26167554 0.         0.         0.         0.         0.
  0.        ]
 [0.16846483 0.15246964 0.         0.         0.         0.
  0.        ]
 [0.26167554 0.         0.         0.         0.         0.
  0.        ]
 [0.26167554 0.         0.         0.         0.         0.
  0.        ]
 [0.26167554 0.         0.         0.         0.         0.
  0.        ]
 [0.26167554 0.         0.         0. 

### Bag of Wordsの実装2
##### 参考URL：https://www.pytry3g.com/entry/2018/03/21/181514#Bag-of-Words%E3%81%A3%E3%81%A6%E4%BD%95

In [15]:
# 形態素解析した単語にidを割り振る
wakati_dic = {}# {単語:id}
for wakati in wakati_list:# wakatiには['先物', 'オプション', '委託', '個人', '売買代金', 'シェア', '高水準', '維持']と1文章すべてに含まれる単語のリストが入っている
    for word in wakati:# wordには「国内」、「経済」、、、など各単語が入っている
        if word in wakati_dic:#もしwordがwakati_dicの中に含まれていれば単語にidを割り振る処理をスキップする(単語の重複を除く)
            continue
        wakati_dic[word] = len(wakati_dic)# key(今回はword)にvalue(今回はid)を割り振る 辞書型の要素数取得にもlen()を使用できる
wakati_dic

{'100%': 64,
 '20': 57,
 '2018年': 52,
 '35.9%': 37,
 '4月': 53,
 '6月': 54,
 'SBI証券': 60,
 'する': 4,
 'なか': 5,
 'よう': 16,
 'オプション': 40,
 'シェア': 35,
 'トップ': 66,
 'トレーディング': 23,
 'ビジネス': 21,
 'ホールセール': 20,
 'マーケット': 6,
 '一進一退': 14,
 '上回る': 33,
 '上場会社': 55,
 '中': 17,
 '中国': 9,
 '他社': 31,
 '信託報酬': 51,
 '個人': 42,
 '先物': 39,
 '前年同期': 27,
 '収益': 24,
 '問題': 11,
 '四半期': 47,
 '回復': 3,
 '国内': 0,
 '増加': 26,
 '増収増益': 29,
 '売買代金': 43,
 '大きい': 32,
 '委託': 41,
 '展開': 15,
 '引受': 61,
 '当社': 18,
 '投資信託': 45,
 '拡大': 22,
 '数': 56,
 '更新': 50,
 '期間': 59,
 '末': 48,
 '業界': 65,
 '業績': 19,
 '残高': 46,
 '比': 28,
 '獲得': 38,
 '率': 63,
 '環境': 7,
 '社': 58,
 '等': 13,
 '米国': 8,
 '経済': 1,
 '維持': 36,
 '緩やか': 2,
 '警戒感': 12,
 '貿易摩擦': 10,
 '過去最高': 49,
 '達成': 30,
 '金融': 25,
 '関与': 62,
 '高い': 34,
 '高水準': 44}

In [16]:
# gensimで特徴語辞書を作成
# 参考URL：https://qiita.com/yasunori/items/31a23eb259482e4824e2
from gensim import corpora

# words はさっきの単語リスト
dictionary = corpora.Dictionary(wakati_list)
print(dictionary.token2id)

{'する': 0, 'なか': 1, 'マーケット': 2, '一進一退': 3, '中国': 4, '問題': 5, '回復': 6, '国内': 7, '展開': 8, '環境': 9, '等': 10, '米国': 11, '経済': 12, '緩やか': 13, '警戒感': 14, '貿易摩擦': 15, 'よう': 16, 'トレーディング': 17, 'ビジネス': 18, 'ホールセール': 19, '中': 20, '前年同期': 21, '収益': 22, '増加': 23, '増収増益': 24, '当社': 25, '拡大': 26, '業績': 27, '比': 28, '達成': 29, '金融': 30, '35.9%': 31, 'シェア': 32, '上回る': 33, '他社': 34, '大きい': 35, '獲得': 36, '維持': 37, '高い': 38, 'オプション': 39, '個人': 40, '先物': 41, '売買代金': 42, '委託': 43, '高水準': 44, '信託報酬': 45, '四半期': 46, '投資信託': 47, '更新': 48, '末': 49, '残高': 50, '過去最高': 51, '20': 52, '2018年': 53, '4月': 54, '6月': 55, '上場会社': 56, '数': 57, '社': 58, '100%': 59, 'SBI証券': 60, 'トップ': 61, '引受': 62, '期間': 63, '業界': 64, '率': 65, '関与': 66}


In [17]:
# BoWの要領で各文章の特徴語をカウントして特徴ベクトル作る
# https://qiita.com/yasunori/items/31a23eb259482e4824e2
for word in wakati_list:
    vec = dictionary.doc2bow(word)
    print(vec)

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1)]
[(9, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 2), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1)]
[(0, 1), (25, 1), (31, 1), (32, 2), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1)]
[(32, 1), (37, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1)]
[(0, 1), (37, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 2), (51, 1)]
[(52, 1), (53, 2), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1)]
[(59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1)]
